In [1]:
from IPython.display import HTML, display

url = "https://github.com/pparkitn/kaggle/blob/0718c0df988ff8e94e8421ba218e07494d3e1682/Images/Customer_Matching.png?raw=true"
display(HTML(f'<img src="{url}" alt="Customer Matching" style="max-width:100%;">'))


# Matching Customers
- This code outlines a process for matching customers from a database based on various string similarity measures and leveraging record linkage techniques. The overall objective is to identify potential duplicate or matching customer records by comparing their details such as names, emails, and addresses.
- Levenshtein: For computing string similarity using the Levenshtein distance.
- fuzzymatcher: A library for record linkage (matching records from different datasets).
- recordlinkage: Another package for record linkage, enabling comparison of potential matching records.

## Key Concepts:
- Record Linkage: The process of identifying records in a dataset that refer to the same entity but are not identical (e.g., duplicate records with slight variations).
- String Similarity Metrics: Measures like Levenshtein distance and token-based ratios are used to quantify how similar two strings are, which is central to the matching process.
- Thresholding and Scoring: Potential matches are scored, and only those above a defined threshold are retained for further analysis.

In summary, this code demonstrates how to use string matching and record linkage techniques to identify and match similar customer records in a dataset, which is useful in applications like data cleaning, deduplication, or fraud detection.

In [2]:
!pip install Levenshtein
!pip install fuzzymatcher
!pip install recordlinkage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13902 sha256=5cadec296576e5288aecd2406e8f0a73940853750705715574691e96cfba2e6c
  Stored in directory: /root/.cache/pip/wheels/23/dd/1d/6cdd346605db62bde1f60954155e9ce48f4681c243f265b704
Successfully built metaphone
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.0/336.0 kB 14.6 MB/s eta 0:00:00


In [3]:
from fuzzywuzzy import fuzz
import math
import pandas as pd
import numpy as np
import glob, os
from datetime import datetime, timedelta
import csv
from tqdm import tqdm
from cryptography.fernet import Fernet
import Levenshtein as lev
import getpass
import base64
import sqlalchemy as sa
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

# Database interaction imports
import csv, sqlite3

import pandas as pd
from pathlib import Path
import fuzzymatcher
import recordlinkage

## Read Data from Database
- The SQLite database file Sonify_DB.db is read, and the Dim_Customers table is loaded into a pandas DataFrame. 
- Only the first 500 rows are selected for simplicity in processing.

In [4]:
db_file = "/kaggle/input/sonify-synthetic-music-service/Sonify_DB.db"

con = sqlite3.connect(db_file) 
Dim_Customers_df = pd.read_sql_query("""SELECT * from Dim_Customers""", con)
con.close()
Dim_Customers_df = Dim_Customers_df.head(500)
Dim_Customers_df.head(1)

,Customer_ID,Country,Province,email,FirstName,LastName
0,100000,Canada,Prince Edward Island,Sarah,Allen,sarah.allen@gmail.com


## String Matching Examples
- Several string matching methods are tested on two example strings ('Peter' and 'Peter2'):
- token_set_ratio: Tokenizes strings and compares the sets.
- Levenshtein ratio: Measures the distance between two strings.
- fuzz.ratio, partial_ratio, and token_sort_ratio: Different comparison methods provided by the fuzzywuzzy library.

In [5]:
str1 = 'Peter'
str2 = 'Peter2'

result = fuzz.token_set_ratio(str1,str2)
print(result)

result = lev.ratio(str1,str2)
print(result)

result = fuzz.ratio(str1,str2)
print(result)

result = fuzz.partial_ratio(str1,str2)
print(result)

result = fuzz.token_sort_ratio(str1,str2)    
print(result)

91
0.9090909090909091
91
100
91


## Record Linkage Indexing
- The recordlinkage.Index() class is used to create pairs of customer records (candidates) to be compared. This step generates pairs of rows from the Dim_Customers_df DataFrame for potential matching.

In [6]:
indexer = recordlinkage.Index()
indexer.full()
candidates = indexer.index(Dim_Customers_df, Dim_Customers_df)
print(len(candidates))

250000


## Comparison of Customer Records
- The Compare class from recordlinkage is used to define a set of comparisons between customer records:
- Exact matches for columns like Province and email.
- String comparisons for columns like FirstName, LastName, and Country, with a threshold of 0.85 for similarity.

In [7]:
%%time

# initialise class
compare = recordlinkage.Compare(n_jobs=-1, non_empty=True)

# initialise similarity measurement algorithms
compare.exact('Province', 'Province', label='Province')
compare.exact('email', 'email', label='email')
compare.string('FirstName','FirstName',threshold=0.85,label='FirstName')
compare.string('LastName','LastName',threshold=0.85,label='LastName')
compare.string('Country','Country', method='levenshtein', threshold=0.85,label='Country')
features = compare.compute(candidates, Dim_Customers_df, Dim_Customers_df)

INFO:recordlinkage:comparing [1/?] - time: 4.38s - pairs: 62500
INFO:recordlinkage:comparing [1/?] - time: 4.38s - pairs_total: 62500
INFO:recordlinkage:comparing [1/?] - time: 4.54s - pairs: 62500
INFO:recordlinkage:comparing [1/?] - time: 4.54s - pairs_total: 62500
INFO:recordlinkage:comparing [1/?] - time: 4.53s - pairs: 62500
INFO:recordlinkage:comparing [1/?] - time: 4.53s - pairs_total: 62500


CPU times: user 95.4 ms, sys: 126 ms, total: 221 ms
Wall time: 7.09 s


INFO:recordlinkage:comparing [1/?] - time: 4.17s - pairs: 62500
INFO:recordlinkage:comparing [1/?] - time: 4.17s - pairs_total: 62500


In [8]:
features

Province  email  FirstName  LastName  Country
0   0           1      1        1.0       1.0      1.0
    1           0      0        0.0       0.0      1.0
    2           0      0        0.0       0.0      1.0
    3           0      0        0.0       0.0      1.0
    4           0      0        0.0       0.0      1.0
...           ...    ...        ...       ...      ...
499 495         0      0        0.0       0.0      1.0
    496         0      0        0.0       0.0      1.0
    497         0      0        0.0       0.0      1.0
    498         0      0        0.0       0.0      1.0
    499         1      1        1.0       1.0      1.0

[250000 rows x 5 columns]

## Examine Similarity Scores
- The features DataFrame contains similarity scores for each pair of records. The code checks how many pairs meet a certain threshold and computes the sum of all feature matches across pairs.

In [9]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

5.0       506
4.0       106
3.0      1194
2.0     31498
1.0    216696
Name: count, dtype: int64

## Identify Potential Matches
- Customer pairs that have a score greater than 1 are considered as potential matches.
- A new Score column is created by summing the individual string similarity scores from different fields (FirstName, LastName, email).

In [10]:
potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches['Score'] = potential_matches['FirstName']+potential_matches['LastName']+potential_matches['email']

In [11]:
potential_matches.head(5)

,level_0,level_1,Province,email,FirstName,LastName,Country,Score
0,0,0,1,1,1.0,1.0,1.0,3.0
1,0,12,1,0,0.0,0.0,1.0,0.0
2,0,16,1,0,0.0,0.0,1.0,0.0
3,0,24,0,0,1.0,0.0,1.0,1.0
4,0,27,1,0,0.0,0.0,1.0,0.0


## Filter Matches Based on Score
- Potential matches with a total score below a threshold (2 in this case) are removed.

In [12]:
# Remove Low Matches
potential_matches = potential_matches[(potential_matches['Score'] >= 2)]
potential_matches

,level_0,level_1,Province,email,FirstName,LastName,Country,Score
0,0,0,1,1,1.0,1.0,1.0,3.0
50,0,359,0,0,1.0,1.0,1.0,2.0
72,1,1,1,1,1.0,1.0,1.0,3.0
137,2,2,1,1,1.0,1.0,1.0,3.0
195,3,3,1,1,1.0,1.0,1.0,3.0
...,...,...,...,...,...,...,...,...
33194,498,282,0,0,1.0,1.0,1.0,2.0
33228,498,498,1,1,1.0,1.0,1.0,3.0
33269,499,270,0,1,0.0,1.0,1.0,2.0
33278,499,334,0,1,1.0,1.0,1.0,3.0


## Merge Data to Retrieve Customer IDs
- The potential_matches DataFrame is merged with the original Dim_Customers_df to retrieve the customer IDs for the matched records.

In [13]:
look_up_0 = potential_matches.merge(Dim_Customers_df, left_on='level_0', right_index=True)
look_up_0 = look_up_0[['level_0','Customer_ID']]
look_up_0.head(5)

,level_0,Customer_ID
0,0,100000
50,0,100000
72,1,100001
137,2,100002
195,3,100003


In [14]:
look_up_1 = potential_matches.merge(Dim_Customers_df, left_on='level_1', right_index=True)
look_up_1 = look_up_1[['level_1','Customer_ID']]
look_up_1.rename(columns={"Customer_ID":"Customer2_ID"}, inplace=True)
look_up_1.head(5)

,level_1,Customer2_ID
0,0,100000
50,359,100359
72,1,100001
137,2,100002
195,3,100003


In [15]:
# Merge df1 on column 'A' and df2 on its index
potential_matches = potential_matches.merge(look_up_0, left_on='level_0', right_on='level_0')
potential_matches = potential_matches.merge(look_up_1, left_on='level_1', right_on='level_1')
potential_matches

,level_0,level_1,Province,email,FirstName,LastName,Country,Score,Customer_ID,Customer2_ID
0,0,0,1,1,1.0,1.0,1.0,3.0,100000,100000
1,0,0,1,1,1.0,1.0,1.0,3.0,100000,100000
2,0,0,1,1,1.0,1.0,1.0,3.0,100000,100000
3,0,0,1,1,1.0,1.0,1.0,3.0,100000,100000
4,0,359,0,0,1.0,1.0,1.0,2.0,100000,100359
...,...,...,...,...,...,...,...,...,...,...
7885,499,499,1,1,1.0,1.0,1.0,3.0,100499,100499
7886,499,499,1,1,1.0,1.0,1.0,3.0,100499,100499
7887,499,499,1,1,1.0,1.0,1.0,3.0,100499,100499
7888,499,499,1,1,1.0,1.0,1.0,3.0,100499,100499


## Final Filtering of Matches
- The code filters out duplicate matches and ensures that the same customer ID is not matched with itself.

In [16]:
matches = potential_matches[(potential_matches['Customer_ID'] != potential_matches['Customer2_ID']) & (potential_matches['Score'] >= 2)]
matches = matches.drop_duplicates()

In [17]:
matches.head(5)

,level_0,level_1,Province,email,FirstName,LastName,Country,Score,Customer_ID,Customer2_ID
4,0,359,0,0,1.0,1.0,1.0,2.0,100000,100359
18,3,113,0,0,1.0,1.0,1.0,2.0,100003,100113
28,4,132,1,1,0.0,1.0,1.0,2.0,100004,100132
37,6,295,0,1,1.0,1.0,1.0,3.0,100006,100295
45,7,331,1,1,0.0,1.0,1.0,2.0,100007,100331


## Display Matched Customer Records
- Finally, the matched customer pairs are displayed, and individual customer records can be retrieved for review.

In [18]:
Dim_Customers_df[Dim_Customers_df['Customer_ID'].isin([100002,100168])]

,Customer_ID,Country,Province,email,FirstName,LastName
2,100002,Canada,Quebec,Alexander,Cooper,alexander.cooper@gmail.com
168,100168,Canada,Manitoba,William,Green,william.green@gmail.com


In [19]:
Dim_Customers_df[Dim_Customers_df['Customer_ID'].isin([100003,100090])]

,Customer_ID,Country,Province,email,FirstName,LastName
3,100003,Canada,Newfoundland and Labrador,James,Reed,james.reed@gmail.com
90,100090,Canada,Manitoba,Henry,Stewart,henry.stewart@gmail.com
